# Data Load & Cleansing
In this notebook, we will load the sample data and investigate the content. We will also cleanse the data. We first import the packages we will be using and then we load the sample data. We are using the `consumer_id` column as the index column for the data frame under the assumption this field contains unique values.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.plotting.register_matplotlib_converters()

file_path = "../data/data.csv"
data = pd.read_csv(file_path, index_col = "consumer_id")

First five rows of the dataframe looks like below. We can easily see that we have some `NaN` values. We will look into these columns with `NaN` values later when we are cleansing the data.

In [2]:
data.head()

,gender,has_gender,has_first_name,has_last_name,has_email,has_dob,customer_age,account_age,account_last_updated,account_status,app_downloads,unique_offer_clicked,total_offer_clicks,unique_offer_rides,total_offer_rides,avg_claims,min_claims,max_claims,total_offers_claimed
consumer_id,,,,,,,,,,,,,,,,,,,
1284b75c-ecae-4015-8e3d-359c0347ede8,NaN,0,1,1,1,0,NaN,188,174,0,1,3,3,8,52,1.0,1,1,1
128af162-d2c3-4fe4-986c-359c8bdc6c04,NaN,0,1,1,1,0,NaN,185,174,0,1,3,8,5,58,1.0,1,1,1
12aada5e-36eb-4e9e-8d62-359c076c1b40,NaN,0,1,1,1,0,NaN,188,174,0,1,3,4,9,183,1.0,1,1,3
12c2e02f-bc79-4048-83ba-359cd3280dcf,NaN,0,1,1,1,0,NaN,184,174,0,1,3,7,4,56,1.0,1,1,1
12fabdf0-0582-489e-a6d3-35509ab8ae6f,f,1,1,1,1,1,22.0,572,173,0,1,4,6,7,83,1.0,1,1,3


Lets separate object and numeric columns.

In [3]:
obj_cols = [col for col in data.columns if data[col].dtype == "object"]
num_cols = [col for col in data.columns if data[col].dtype in ["int64", "float64"]]

We can see that the only object column is the `gender` column.

In [4]:
print(num_cols, obj_cols)

['has_gender', 'has_first_name', 'has_last_name', 'has_email', 'has_dob', 'customer_age', 'account_age', 'account_last_updated', 'account_status', 'app_downloads', 'unique_offer_clicked', 'total_offer_clicks', 'unique_offer_rides', 'total_offer_rides', 'avg_claims', 'min_claims', 'max_claims', 'total_offers_claimed'] ['gender']


Let's also get those columns with `NaN` values. It looks like there are only two columns with `NaN` values.

In [5]:
obj_cols_with_na = [col for col in obj_cols if data[col].isnull().any()]
num_cols_with_na = [col for col in num_cols if data[col].isnull().any()]

print(obj_cols_with_na, num_cols_with_na)

['gender'] ['customer_age']


We define the following function to get the percent of number of `NaN` values in those columns with `NaN` values.

In [6]:
def na_ratio(col):
    ratio = col.isnull().sum() / len(col)
    if (ratio > 0):
        return ratio
    
data[obj_cols_with_na + num_cols_with_na].apply(na_ratio)

gender          0.4522
customer_age    0.5936
dtype: float64

These ratios are rather high. And, there is almost no way to figure out the missing data. Imputation would remove most of the data. Since we have two columns (`has_gender` and `has_dob`) that indicate if the record has a gender or age, we will be dropping the `gender` and `customer_age` columns. In addition, we will also drop `account_status` column as it has only one single value.

In [7]:
data.drop(obj_cols_with_na + num_cols_with_na + ["account_status"], axis = 1, inplace = True)

data.head()

,has_gender,has_first_name,has_last_name,has_email,has_dob,account_age,account_last_updated,app_downloads,unique_offer_clicked,total_offer_clicks,unique_offer_rides,total_offer_rides,avg_claims,min_claims,max_claims,total_offers_claimed
consumer_id,,,,,,,,,,,,,,,,
1284b75c-ecae-4015-8e3d-359c0347ede8,0,1,1,1,0,188,174,1,3,3,8,52,1.0,1,1,1
128af162-d2c3-4fe4-986c-359c8bdc6c04,0,1,1,1,0,185,174,1,3,8,5,58,1.0,1,1,1
12aada5e-36eb-4e9e-8d62-359c076c1b40,0,1,1,1,0,188,174,1,3,4,9,183,1.0,1,1,3
12c2e02f-bc79-4048-83ba-359cd3280dcf,0,1,1,1,0,184,174,1,3,7,4,56,1.0,1,1,1
12fabdf0-0582-489e-a6d3-35509ab8ae6f,1,1,1,1,1,572,173,1,4,6,7,83,1.0,1,1,3
